In [1]:
!nvidia-smi

Thu Feb 17 15:40:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.30       Driver Version: 462.30       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 165... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   43C    P8     4W /  N/A |    134MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import  Dense, Bidirectional, LSTM, Dropout,Input, Embedding
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model 
from tensorflow.keras.metrics import Precision, AUC, binary_accuracy, TrueNegatives, TruePositives, FalseNegatives, FalsePositives
from helper_functions import mcc_metric
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from transformers import AutoTokenizer

In [3]:
data = pd.read_table('F:/bbb/data/B3DB/B3DB_classification.tsv')
data.head(1)

,NO.,compound_name,IUPAC_name,SMILES,CID,logBB,BBB+/BBB-,Inchi,threshold,reference,group,comments
0,1,sulphasalazine,2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,5339.0,-2.69,BBB-,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NaN,R2|R2|R25|R46|,A,NaN


In [4]:
X = data["SMILES"]

target=data['BBB+/BBB-'].values.tolist()
label_encoder = LabelEncoder()
y_classification = label_encoder.fit_transform(target)

In [5]:
tokenizer = AutoTokenizer.from_pretrained('DeepChem/ChemBERTa-77M-MLM', from_pt = True)

In [6]:
smiles_data = tokenizer(list(X),  truncation=True, padding=True)
smiles_data = smiles_data['input_ids']

In [7]:
xtrain, xtest, ytrain, ytest = train_test_split(smiles_data, y_classification, test_size=0.2, random_state=42)
xtrain, xvalid, ytrain, yvalid  = train_test_split(xtrain, ytrain, test_size= 0.16, random_state=42)

In [8]:
len(y_classification)

7807

In [9]:
np.unique(y_classification, return_counts=True)

(array([0, 1], dtype=int64), array([4956, 2851], dtype=int64))

In [10]:
class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(ytrain), 
            ytrain)

F:\anaconda\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[0 1 0 ... 0 1 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [11]:
class_weights = {0:0.78950695,
                1: 1.36353712} 

In [12]:
word_in_vocab = tokenizer.vocab
len(word_in_vocab)

593

In [13]:
np.array(xtrain[0]).shape

(244,)

In [14]:
input = Input(shape = (244,))

x = Embedding(input_dim=len(word_in_vocab),
                        output_dim=128, 
                        input_length=None,
                        )(input)
x = Bidirectional(LSTM(128, return_sequences=True,name = 'lstm1'))(x)
x = Bidirectional(LSTM(128, return_sequences=True,name = 'lstm1'))(x)
x = Dropout(0.2)(x)
x = Bidirectional(LSTM(128, return_sequences=False,name = 'lstm1'))(x)
x = Dropout(0.2)(x)

x = Dense(256, activation = 'relu')(x)
x = Dense(10, activation = 'relu')(x)
output = Dense(1, activation = "sigmoid")(x)
model = Model(inputs = input, outputs = output)


In [15]:
def save(i):
    return 'F:/bbb/model/nlp/model_'+str(i)+'.tf'

checkpoint = tf.keras.callbacks.ModelCheckpoint(save(33), 
                monitor='val_auc', verbose=1, 
                save_best_only=True, save_weights_only = False, mode='max')
callbacks_list = [checkpoint]

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001) ,loss = BinaryCrossentropy(), metrics = [binary_accuracy, AUC(), mcc_metric 
                    ,Precision(),TrueNegatives(), TruePositives(), FalseNegatives(), FalsePositives()])
model.fit(x = np.array(xtrain), y = np.array(ytrain), validation_data = (np.array(xvalid), np.array(yvalid)), epochs=60, batch_size = 32, 
              callbacks = callbacks_list, class_weight = class_weights)


Epoch 1/60
164/164 [==============================] - ETA: 0s - loss: 0.5486 - binary_accuracy: 0.7455 - auc: 0.7883 - mcc_metric: nan - precision: 0.6486 - true_negatives: 2618.0000 - true_positives: 1292.0000 - false_negatives: 635.0000 - false_positives: 700.0000
Epoch 00001: val_auc improved from -inf to 0.83727, saving model to F:/bbb/model/nlp\model_33.tf


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


164/164 [==============================] - 55s 280ms/step - loss: 0.5486 - binary_accuracy: 0.7455 - auc: 0.7883 - mcc_metric: nan - precision: 0.6486 - true_negatives: 2618.0000 - true_positives: 1292.0000 - false_negatives: 635.0000 - false_positives: 700.0000 - val_loss: 0.4581 - val_binary_accuracy: 0.8070 - val_auc: 0.8373 - val_mcc_metric: 0.5547 - val_precision: 0.7796 - val_true_negatives: 570.0000 - val_true_positives: 237.0000 - val_false_negatives: 126.0000 - val_false_positives: 67.0000
Epoch 2/60
164/164 [==============================] - ETA: 0s - loss: 0.4977 - binary_accuracy: 0.7918 - auc: 0.8260 - mcc_metric: 0.5507 - precision: 0.7248 - true_negatives: 2807.0000 - true_positives: 1346.0000 - false_negatives: 581.0000 - false_positives: 511.0000 ETA: 8s - loss: 0.5015 - binary_accuracy: 0.7878 - auc: 0.8239 - mcc_metric: 0.5448 - precision
Epoch 00002: val_auc improved from 0.83727 to 0.85191, saving model to F:/bbb/model/nlp\model_33.tf


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


...

INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


164/164 [==============================] - 47s 285ms/step - loss: 0.2598 - binary_accuracy: 0.8801 - auc: 0.9584 - mcc_metric: 0.7528 - precision: 0.7980 - true_negatives: 2878.0000 - true_positives: 1738.0000 - false_negatives: 189.0000 - false_positives: 440.0000 - val_loss: 0.3652 - val_binary_accuracy: 0.8320 - val_auc: 0.9261 - val_mcc_metric: 0.6661 - val_precision: 0.7181 - val_true_negatives: 511.0000 - val_true_positives: 321.0000 - val_false_negatives: 42.0000 - val_false_positives: 126.0000
Epoch 27/60
164/164 [==============================] - ETA: 0s - loss: 0.2502 - binary_accuracy: 0.8812 - auc: 0.9613 - mcc_metric: 0.7618 - precision: 0.7860 - true_negatives: 2830.0000 - true_positives: 1792.0000 - false_negatives: 135.0000 - false_positives: 488.0000
Epoch 00027: val_auc did not improve from 0.92613
164/164 [==============================] - 18s 109ms/step - loss: 0.2502 - binary_accuracy: 0.8812 - auc: 0.9613 - mcc_metric: 0.7618 - precision: 0.7860 - true_negatives: 

INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


164/164 [==============================] - 43s 265ms/step - loss: 0.2119 - binary_accuracy: 0.9037 - auc: 0.9724 - mcc_metric: 0.8020 - precision: 0.8332 - true_negatives: 2962.0000 - true_positives: 1778.0000 - false_negatives: 149.0000 - false_positives: 356.0000 - val_loss: 0.3578 - val_binary_accuracy: 0.8370 - val_auc: 0.9359 - val_mcc_metric: 0.6761 - val_precision: 0.7304 - val_true_negatives: 520.0000 - val_true_positives: 317.0000 - val_false_negatives: 46.0000 - val_false_positives: 117.0000
Epoch 30/60
164/164 [==============================] - ETA: 0s - loss: 0.1969 - binary_accuracy: 0.9153 - auc: 0.9759 - mcc_metric: 0.8239 - precision: 0.8506 - true_negatives: 3002.0000 - true_positives: 1799.0000 - false_negatives: 128.0000 - false_positives: 316.0000
Epoch 00030: val_auc did not improve from 0.93585
164/164 [==============================] - 17s 106ms/step - loss: 0.1969 - binary_accuracy: 0.9153 - auc: 0.9759 - mcc_metric: 0.8239 - precision: 0.8506 - true_negatives: 

In [16]:
model.load_weights(save(33))
model.evaluate(np.array(xtest), np.array(ytest))

49/49 [==============================] - 2s 40ms/step - loss: 0.4086 - binary_accuracy: 0.8284 - auc: 0.9175 - mcc_metric: 0.6441 - precision: 0.7183 - true_negatives: 812.0000 - true_positives: 482.0000 - false_negatives: 79.0000 - false_positives: 189.0000


[0.4085613787174225,
 0.8284251093864441,
 0.917498767375946,
 0.6441357731819153,
 0.7183308601379395,
 812.0,
 482.0,
 79.0,
 189.0]

In [17]:
model.evaluate(np.array(xvalid), np.array(yvalid))

32/32 [==============================] - 1s 38ms/step - loss: 0.3578 - binary_accuracy: 0.8370 - auc: 0.9359 - mcc_metric: 0.6761 - precision: 0.7304 - true_negatives: 520.0000 - true_positives: 317.0000 - false_negatives: 46.0000 - false_positives: 117.0000


[0.3578473627567291,
 0.8370000123977661,
 0.93585205078125,
 0.6760877966880798,
 0.7304147481918335,
 520.0,
 317.0,
 46.0,
 117.0]

In [18]:
tf.__version__

'2.7.0'